In [1]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from keras.utils import np_utils

import nltk
from nltk.corpus import stopwords

import pandas as pd
import numpy as np

2022-02-27 17:25:32.380495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-27 17:25:32.380568: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Classifica proposições legislativas

Foi verificado que as proposições do conjunto de dados “proposições legislativas” não estão classificadas de acordo com o conjunto de dados “temas”, então o modelo de apredizado de máquina criado nesse capítulo será com objetivo de classificar tais itens a partir da variável “ementa”.

In [2]:
df_proposicoes = pd.read_csv("dados/proposicoes_legislativas_limpas_vocabulario.csv")
df_proposicoes_classificado = df_proposicoes.dropna(subset=["temas"])
df_proposicoes_classificado = df_proposicoes_classificado[["ementa","temas"]]

In [3]:
df_proposicoes_classificado.shape

(20210, 2)

In [4]:
df_proposicoes_classificado.head()

,ementa,temas
62,PROIBE O EMPREGO DE EXPRESSÕES DESAIROSAS AO H...,Direitos Humanos e Minorias
67,"ESTABELECE O REGIME DE APOSENTADORIA ESPECIAL,...",Trabalho e Emprego
71,"REVOGA A LEI 3841, DE 15 DE DEZEMBRO DE 1960, ...",Trabalho e Emprego
81,DEFINE O PEQUENO PRODUTOR E DA OUTRAS PROVIDEN...,Trabalho e Emprego
96,TRANSFORMA EM APOSENTADORIAS AS DEMISSÕES DE S...,Trabalho e Emprego


Estabelecer "ementa" como variável preditora e "temas" como variável de resposta.

In [5]:
sentences = df_proposicoes_classificado['ementa'].values

Estabelece encode para a variável de resposta

In [6]:
le = preprocessing.LabelEncoder()
le.fit(df_proposicoes_classificado['temas'].unique())

y = le.transform(df_proposicoes_classificado['temas'])

Separar conjunto de treino e teste

In [7]:
sentences_train, sentences_test, y_train, y_test = train_test_split(
   sentences, y, test_size=0.25, random_state=1000)

Converter das palavras das ementas do conjuntos de dados de treino e teste em vetores eliminando stopwords

In [8]:
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)
X_train

hasher = HashingVectorizer(
            n_features=10000,
            stop_words=stopwords.words('portuguese'),
            alternate_sign=False,
            norm=None,
        )
hasher.fit(sentences_train)
X_train_hasher = hasher.transform(sentences_train)
X_test_hasher = hasher.transform(sentences_test)

In [9]:
X_train_hasher.shape

(15157, 10000)

Criar e treinar modelo de classificação a partir do algoritmo Random Forest Classifier

In [10]:
clf = RandomForestClassifier(n_estimators=200,random_state=0)
clf.fit(X_train_hasher, y_train)

RandomForestClassifier(n_estimators=200, random_state=0)

Verificar o coeficiente de determinação (R²), acurácia.

In [11]:
score = clf.score(X_test_hasher, y_test)

print("Acurácia:", score)

Acurácia: 0.7920047496536711


Avaliar modelo qualitativamente

In [12]:
df_random_forest_results = pd.DataFrame([sentences_test,le.inverse_transform(clf.predict(X_test_hasher))]).transpose().rename(columns={0:"ementa",1:"tema"})
df_random_forest_results.head()

,ementa,tema
0,"Altera a Lei nº 10.098, de 19 de dezembro de 2...",Direito Civil e Processual Civil
1,"Altera o art. 6º da Lei nº 10.826, de 22 de de...",Direito Penal e Processual Penal
2,"AUTORIZA O PODER EXECUTIVO A ABRIR, PELO MINIS...",Direitos Humanos e Minorias
3,Cria mecanismos para coibir a violência e a di...,Direitos Humanos e Minorias
4,ASSEGURA AOS FUNCIONARIOS PUBLICOS CIVIS DA UN...,Trabalho e Emprego


Criar lista com probabilidades de classificação das propostas em cada tema

In [13]:
predicted_probabilities = clf.predict_proba(X_test_hasher)

Selecionar o tema com maior probabilidade para cada ementa

In [14]:
df_random_forest_results["probabilidade_predicao"] = np.amax(predicted_probabilities,axis=1) # colocar if max maior do que 0.8

In [15]:
df_random_forest_results.head()

,ementa,tema,probabilidade_predicao
0,"Altera a Lei nº 10.098, de 19 de dezembro de 2...",Direito Civil e Processual Civil,0.865
1,"Altera o art. 6º da Lei nº 10.826, de 22 de de...",Direito Penal e Processual Penal,1.000
2,"AUTORIZA O PODER EXECUTIVO A ABRIR, PELO MINIS...",Direitos Humanos e Minorias,0.945
3,Cria mecanismos para coibir a violência e a di...,Direitos Humanos e Minorias,0.960
4,ASSEGURA AOS FUNCIONARIOS PUBLICOS CIVIS DA UN...,Trabalho e Emprego,0.925


Criar dataframe comparativo entre temas pré-estabelecidos e os classificados pelo modelo

In [16]:
df_ementas_test = pd.DataFrame([sentences_test,le.inverse_transform(y_test)]).transpose().rename(columns={0:"ementa",1:"tema"})

In [17]:
df_ementas_test.head()

,ementa,tema
0,"Altera a Lei nº 10.098, de 19 de dezembro de 2...",Direito Civil e Processual Civil
1,"Altera o art. 6º da Lei nº 10.826, de 22 de de...",Direito Penal e Processual Penal
2,"AUTORIZA O PODER EXECUTIVO A ABRIR, PELO MINIS...",Direitos Humanos e Minorias
3,Cria mecanismos para coibir a violência e a di...,Direitos Humanos e Minorias
4,ASSEGURA AOS FUNCIONARIOS PUBLICOS CIVIS DA UN...,Trabalho e Emprego


In [18]:
df_avaliacao = df_random_forest_results.merge(df_ementas_test,left_on="ementa",right_on="ementa",suffixes=["_resposta_modelo","_correto"])
df_avaliacao["modelo_acertou"] = df_avaliacao["tema_resposta_modelo"] == df_avaliacao["tema_correto"]
df_avaliacao["modelo_acertou"] = df_avaliacao["modelo_acertou"].replace({True: "Sim", False: "Não"})

In [19]:
df_avaliacao["modelo_acertou"].value_counts()

Sim    4148
Não    1079
Name: modelo_acertou, dtype: int64

In [20]:
df_avaliacao[df_avaliacao["probabilidade_predicao"] >= 0.85]["modelo_acertou"].value_counts()

Sim    1696
Não      58
Name: modelo_acertou, dtype: int64

In [21]:
df_avaliacao.head()

,ementa,tema_resposta_modelo,probabilidade_predicao,tema_correto,modelo_acertou
0,"Altera a Lei nº 10.098, de 19 de dezembro de 2...",Direito Civil e Processual Civil,0.865,Direito Civil e Processual Civil,Sim
1,"Altera o art. 6º da Lei nº 10.826, de 22 de de...",Direito Penal e Processual Penal,1.000,Direito Penal e Processual Penal,Sim
2,"AUTORIZA O PODER EXECUTIVO A ABRIR, PELO MINIS...",Direitos Humanos e Minorias,0.945,Direitos Humanos e Minorias,Sim
3,Cria mecanismos para coibir a violência e a di...,Direitos Humanos e Minorias,0.960,Direitos Humanos e Minorias,Sim
4,ASSEGURA AOS FUNCIONARIOS PUBLICOS CIVIS DA UN...,Trabalho e Emprego,0.925,Trabalho e Emprego,Sim


In [22]:
df_ementas_test.tema.value_counts()

Direitos Humanos e Minorias                    2334
Trabalho e Emprego                             1344
Saúde                                           333
Arte, Cultura e Religião                        265
Direito Penal e Processual Penal                191
Previdência e Assistência Social                163
Educação                                        140
Direito Civil e Processual Civil                121
Agricultura, Pecuária, Pesca e Extrativismo      86
Cidades e Desenvolvimento Urbano                 39
Relações Internacionais e Comércio Exterior      16
Viação, Transporte e Mobilidade                  13
Meio Ambiente e Desenvolvimento Sustentável       6
Homenagens e Datas Comemorativas                  1
Economia                                          1
Name: tema, dtype: int64

In [23]:
df_avaliacao.to_csv('dados/avaliacao-qualitativa-modelo-classificacao.csv')

## Aplicando o modelo para classificar todas as proposicoes legislativas

In [24]:
df_proposicoes_total = df_proposicoes[["ementa","temas"]]

In [25]:
ementas = df_proposicoes_total['ementa'].values

In [26]:
ementas_hasher = hasher.transform(ementas)

Aplica o modelo de classificacao em todas as proposições legislativas

In [27]:
df_proposicoes_total_classificadas = pd.DataFrame([ementas,le.inverse_transform(clf.predict(ementas_hasher))]).transpose().rename(
    columns={0:"ementa",1:"temas"})

In [28]:
df_proposicoes_total_classificadas.head()

,ementa,temas
0,MANTEM DECISÃO DENEGATORIA DO TRIBUNAL DE CONT...,Direitos Humanos e Minorias
1,MANTEM ATO DO TRIBUNAL DE CONTAS DA UNIÃO DE R...,Direitos Humanos e Minorias
2,MANTÉM DECISÃO DENEGATÓRIA DO TRIBUNAL DE CONT...,Direitos Humanos e Minorias
3,MANTÉM ATO DO TRIBUNAL DE CONTAS DA UNIÃO QUE ...,Direitos Humanos e Minorias
4,MANTEM O ATO DO TRIBUNAL DE CONTAS DA UNIÃO QU...,Direitos Humanos e Minorias


In [29]:
df_proposicoes_total_classificadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152386 entries, 0 to 152385
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ementa  152386 non-null  object
 1   temas   152386 non-null  object
dtypes: object(2)
memory usage: 2.3+ MB


Informar a probabilidade de acerto de cada tema

In [30]:
temas_probabilities = clf.predict_proba(ementas_hasher)

In [31]:
df_proposicoes_total_classificadas["probabilidade_predicao"] = np.amax(temas_probabilities, axis=1)

In [32]:
df_proposicoes_total_classificadas.head()

,ementa,temas,probabilidade_predicao
0,MANTEM DECISÃO DENEGATORIA DO TRIBUNAL DE CONT...,Direitos Humanos e Minorias,0.7600
1,MANTEM ATO DO TRIBUNAL DE CONTAS DA UNIÃO DE R...,Direitos Humanos e Minorias,0.5050
2,MANTÉM DECISÃO DENEGATÓRIA DO TRIBUNAL DE CONT...,Direitos Humanos e Minorias,0.7750
3,MANTÉM ATO DO TRIBUNAL DE CONTAS DA UNIÃO QUE ...,Direitos Humanos e Minorias,0.7150
4,MANTEM O ATO DO TRIBUNAL DE CONTAS DA UNIÃO QU...,Direitos Humanos e Minorias,0.4725


In [33]:
df_proposicoes_total_classificadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152386 entries, 0 to 152385
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ementa                  152386 non-null  object 
 1   temas                   152386 non-null  object 
 2   probabilidade_predicao  152386 non-null  float64
dtypes: float64(1), object(2)
memory usage: 3.5+ MB


Limpa temas cuja a probabilidade de acerto é menor do que 85%

In [34]:
def retira_tema_com_baixa_probabilidade_acerto(proposicoes):
        if proposicoes['probabilidade_predicao'] >= 0.85:
            return proposicoes['temas']
        else:
            return np.nan

In [35]:
df_proposicoes_total_classificadas['temas'] = df_proposicoes_total_classificadas.apply(retira_tema_com_baixa_probabilidade_acerto, 
                                                                                      axis=1)

Reunir conjunto de dados de proposições legislativas com classificação realizada

In [36]:
df_proposicoes_classificador = df_proposicoes.join(df_proposicoes_total_classificadas, rsuffix='_classificador')

In [37]:
df_proposicoes_classificador.shape

(152386, 12)

In [38]:
df_proposicoes_classificador.head()

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords,temas,ementa_classificador,temas_classificador,probabilidade_predicao
0,168293,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM DECISÃO DENEGATORIA DO TRIBUNAL DE CONT...,NaN,"manutenção,decisão,tribunal,contas,união,tcu,d...",NaN,MANTEM DECISÃO DENEGATORIA DO TRIBUNAL DE CONT...,NaN,0.7600
1,168297,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM ATO DO TRIBUNAL DE CONTAS DA UNIÃO DE R...,NaN,"manutenção,ato,tribunal,contas,união,tcu,recus...",NaN,MANTEM ATO DO TRIBUNAL DE CONTAS DA UNIÃO DE R...,NaN,0.5050
2,168300,PDC,1965,135,Projeto de Decreto Legislativo,MANTÉM DECISÃO DENEGATÓRIA DO TRIBUNAL DE CONT...,NaN,"manutenção,decisão,tribunal,contas,união,tcu,d...",NaN,MANTÉM DECISÃO DENEGATÓRIA DO TRIBUNAL DE CONT...,NaN,0.7750
3,168303,PDC,1965,135,Projeto de Decreto Legislativo,MANTÉM ATO DO TRIBUNAL DE CONTAS DA UNIÃO QUE ...,NaN,"manutenção,ato,tribunal,contas,união,tcu,negaç...",NaN,MANTÉM ATO DO TRIBUNAL DE CONTAS DA UNIÃO QUE ...,NaN,0.7150
4,168307,PDC,1965,135,Projeto de Decreto Legislativo,MANTEM O ATO DO TRIBUNAL DE CONTAS DA UNIÃO QU...,NaN,"manutenção,ato,tribunal,contas,união,tcu,deneg...",NaN,MANTEM O ATO DO TRIBUNAL DE CONTAS DA UNIÃO QU...,NaN,0.4725


In [39]:
df_proposicoes_classificador.drop(columns=['temas', 'ementa_classificador', 'probabilidade_predicao'], inplace=True)

In [40]:
df_proposicoes_classificador.to_csv('dados/proposicoes_legislativas_limpas_classificadas.csv', index=False)